In [46]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import warnings
import json
warnings.filterwarnings('ignore')

In [24]:
tweet_df = pd.read_csv('Datasets/data.csv',encoding="ISO-8859-1")
t_df = pd.read_csv('Datasets/data1.csv',encoding="ISO-8859-1")
tweet_df.drop(tweet_df.columns.difference(['created_at','full_text']),axis=1, inplace=True)
t_df.drop(t_df.columns.difference(['created_at','tweet']),axis=1, inplace=True)
tweet_df = tweet_df.rename(columns={'full_text': 'tweet'})
frames=[tweet_df, t_df]
final_df = pd.concat(frames, ignore_index=True)
final_df

,created_at,tweet
0,2022-07-13T14:29:17.000Z,PNP OIC PLTGEN Danao Jr. confirmed that two me...
1,2022-07-13T12:10:52.000Z,This anti-illegal drug operation also led to t...
2,2022-07-13T11:20:38.000Z,A regional level most wanted person for illega...
3,2022-07-12T13:18:14.000Z,Three former members of a CTG in Masbate are n...
4,2022-07-12T11:53:06.000Z,A 37-year-old drug suspect was busted in an an...
...,...,...
1485,2021-09-02 12:12:11 Malay Peninsula Standard Time,ELEAZAR: 92% OF PNP PERSONNEL RECEIVED VACCINE...
1486,2021-09-01 14:35:45 Malay Peninsula Standard Time,ELEAZAR: POLICEMEN CHARGED WITH MURDER BY DOJ ...
1487,2021-09-01 12:41:19 Malay Peninsula Standard Time,POLICE CONDUCT MOBILE LIBRARY FOR CHILDREN IN ...
1488,2021-09-01 12:09:47 Malay Peninsula Standard Time,IAS RECOMMENDATION ON POLICEMAN INVOLVED IN FA...


In [65]:
datasets = []
datasets.append(pd.read_csv('Datasets/combine/data.csv',encoding="ISO-8859-1").tweet)
datasets.append(pd.read_csv('Datasets/combine/data1.csv',encoding="ISO-8859-1").tweet)
datasets.append(pd.read_csv('Datasets/combine/data2.csv',encoding="ISO-8859-1").tweet)
datasets.append(pd.read_csv('Datasets/combine/data3.csv',encoding="ISO-8859-1").tweet)
datasets.append(pd.read_csv('Datasets/combine/data4.csv',encoding="ISO-8859-1").tweet)
datasets.append(pd.read_csv('Datasets/combine/data5.csv',encoding="ISO-8859-1").tweet)
datasets.append(pd.read_csv('Datasets/combine/data6.csv',encoding="ISO-8859-1").tweet)
datasets.append(pd.read_csv('Datasets/combine/data7.csv',encoding="ISO-8859-1").tweet)
final_df = pd.concat([dataset for dataset in datasets], axis=0)
final_df.to_csv("data16K.csv")
final_df

0       PNP OIC PLTGEN Danao Jr. confirmed that two me...
1       This anti-illegal drug operation also led to t...
2       A regional level most wanted person for illega...
3       Three former members of a CTG in Masbate are n...
4       A 37-year-old drug suspect was busted in an an...
                              ...                        
6506    DepEd strives to enhance reading skills of stu...
6507    OVP taking ISO certification as challenge to f...
6508    The shooting stemmed from the heated argument ...
6509    AFP engineers clear 30% of Marawi areas from e...
6510    10 US citizens, 2 locals killed in Costa Rica ...
Name: tweet, Length: 16763, dtype: object

In [70]:
#final_df = pd.read_csv('Datasets/final.csv',encoding="ISO-8859-1")
tweets_to_keep = ['murder','assassinate','massacre','massacred','drug',
                  'shabu','drugs','cppnpa','ctg','terrorist','murdered',
                  'terrorism','homicide','regicide','butcher','resisted',
                  'resistance','smuggling','smuggle','smuggled','theft',
                  'robbery','rape','carnapping','assault','marijuana',
                  'private armed group','buy-bust','kidnap','kidnapping',
                  'battery','sexual assault','molest','molestation',
                  'defilement','deflowering','hijack','hijacking',
                  'abduction','mugging','steal','stealing','thieving',
                  'robbing','shoplift','shoplifting','holdup','mugging',
                  'contraband','export','arrest','arrested','busted','fraud',
                  'fraudulent','scam','counterfeit','forgery']

final_df = final_df.drop_duplicates()
final_df = final_df.reset_index()

In [72]:
import re
from nltk.tokenize import word_tokenize
tagalog_stops = pd.read_csv("Datasets/tagalog_stopwords.csv").a.to_list()
stop_words = stopwords.words('english') + tagalog_stops
def clean_tweets(text):
    text = text.lower()
    text = re.sub(r'RT @[\w]*','',str(text))
    text = re.sub(r'@[\w]*','',str(text))
    text = re.sub(r'#([a-zA-Z0-9_]{1,50})','',str(text))
    text = re.sub(r'https?://[A-Za-z0-9./]*','',str(text))
    text = re.sub(r'\n','',str(text))
    text = re.sub(r'[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', u'', str(text))
    text = re.sub(r'[^\x00-\x7f]', '', str(text))
    text = re.sub(r'[^\w\s]', '', str(text))
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)
final_df['tweet'] = final_df['tweet'].apply(lambda x: clean_tweets(x))
final_df

,index,tweet
0,0,pnp oic pltgen danao jr confirmed two members ...
1,1,antiillegal drug operation also led seizure 25...
2,2,regional level wanted person illegal drugs nab...
3,3,three former members ctg masbate reintegrated ...
4,4,37yearold drug suspect busted antiillegal drug...
...,...,...
15326,6506,deped strives enhance reading skills students
15327,6507,ovp taking iso certification challenge improve...
15328,6508,shooting stemmed heated argument isagani anche...
15329,6509,afp engineers clear 30 marawi areas explosives


In [73]:
json_city = json.load(open("Datasets/cities.geojson", "r"))
id_map = {}
city_map = {}
province_map = {}
region_map = {}
index = 0
for feature in json_city["features"]:
    #feature["id"] = feature["properties"]["ID_2"]
    feature["id"] = index
    if feature["geometry"] != None:
        id_map[feature["properties"]["NAME_1"].lower() + " " + feature["properties"]["NAME_2"].lower()] = feature["id"]
        words = feature["properties"]["NAME_2"].lower().split()
        for w in words:
            city_map[w] = feature["id"]
        words = feature["properties"]["NAME_1"].lower().split()
        for w in words:
            province_map[w] = feature["id"]
        region_map[feature["properties"]["REGION"].lower().split("(")[0].strip()] = feature["id"]
    index += 1

In [74]:
from nltk import ngrams
final_df['coords'] = np.NaN
#no_coords = final_df[['tweet','coords','type']]
no_coords = final_df[['tweet','coords']]

def locate_tweet(text):
    found = False
    n_gram = 1
    while not found:
        # spliting the sentence
        words = ngrams(text.split(), n_gram)
        result = ""
        text_tokens = word_tokenize(text)
        city = ""
        province = ""
        init_loc = ""
        for w in words:
            w = re.sub(r'[^a-zA-Z]', '', str(w))
            if city_map.__contains__(w) and city == "":
                city = w
                init_loc = w + " " + init_loc
            if province_map.__contains__(w) and province == "":
                province = w
                init_loc = init_loc  + w
            if city != "" and province != "":
                init_loc.strip()
                break;
        index_loc = -1
        if id_map.__contains__(init_loc):
            index_loc = id_map[init_loc]
        elif province != "":
            index_loc = province_map[province]
        elif city != "":
            index_loc = city_map[city]
        else:
            if n_gram == 3:
                return np.NaN
            n_gram += 1
        if index_loc == -1 and n_gram == 3:
            return np.NaN
        sz = len(json_city["features"][int(index_loc)]["geometry"]["coordinates"])
        sz2 = json_city["features"][int(index_loc)]["geometry"]["coordinates"][np.random.randint(sz)]
        loc = sz2[np.random.randint(len(sz2))]
        region = json_city["features"][int(index_loc)]["properties"]["REGION"]
        location = re.findall('\\d+.\\d+', str(loc))
        if loc is not None:
            found = True
    return str(float(location[1])+-np.random.randint(2))+"/"+str(float(location[0])+0.25)+":"+init_loc+":"+region

no_coords['coords'] = no_coords['tweet'].apply(locate_tweet)

regions = json.load(open("Datasets/Regions.json", "r"))

# Add Names
def add_location_name(text):
    return text.split(":")[1]
def add_region(text):
    return text.split(":")[2]

no_coords['loc_name'] = no_coords['coords'].apply(add_location_name)
no_coords['region_name'] = no_coords['coords'].apply(add_region)
no_coords = no_coords.loc[no_coords['loc_name'] != ""]
# Preprocess
def preprocess_coords(coords):
    numbers = re.findall('\\d+', coords)
    return numbers[2]+"."+numbers[3]+"/"+numbers[0]+"."+numbers[1]

no_coords['coords'] = no_coords['coords'].apply(preprocess_coords)

no_coords

,tweet,coords,loc_name,region_name
0,pnp oic pltgen danao jr confirmed two members ...,124.386787/12.0354,danao sorsogon,Bicol Region (Region V)
3,three former members ctg masbate reintegrated ...,123.931534/11.18135,masbate masbate,Bicol Region (Region V)
4,37yearold drug suspect busted antiillegal drug...,123.642082/7.73305,san,Zamboanga Peninsula (Region IX)
7,police arrested monday member communist terror...,122.032257/17.68758,cagayan cagayan,Cagayan Valley (Region II)
8,joint elements tampakan mps piu 1st pmfc scppo...,125.406998/5.46368,tampakan,SOCCSKSARGEN (Region XII)
...,...,...,...,...
15312,movenpick hotel soon rise quezon city,122.386963/14.07083,quezon quezon,CALABARZON (Region IV-A)
15315,nfl del rio pagano fired,123.637917/7.104139999999999,del del,Zamboanga Peninsula (Region IX)
15320,north korea may join winter olympics,125.117737/6.86882,north,SOCCSKSARGEN (Region XII)
15322,thai companies invest baguio,120.819122/15.370180000000001,baguio,Cordillera Administrative Region (CAR)


Text Classification

In [75]:
word_map = {
    'murder': ['murder','assassinate','massacre','homicide','regicide','butcher','robberyslay', 'killed'],
    'physical injury': ['assault','battery','mugging','manhandling'],
    'kidnapping': ['kidnap','kidnapping','abduction','abductors','kidnapped'],
    'rape':['rape','sexual assault','molest','molestation','defilement','deflowering','raping','rapist','harrassing','harrass','nude'],
    'robbery':['robbery','theft','steal','stealing','thieving','robbing','shoplift','shoplifting','holdup','robberyextortion','extortion', 'extortionist'],
    'carnapping':['carnapping','hijacking','hijack'],
    'drug':['drug','shabu','drugs','marijuana','buybust','9165', 'pusher', 'dealer'],
    'terrorism':['cppnpa','cppnpandf','ctg','terrorist','terrorists','terrorism','private','syndicate','abu sayyaf', 'extremist'],
    'obstruction':['resistance','resisted','obstruction'],
    'smuggling':['smuggling','smuggle','smuggled','contraband','export','fugitive','fugitives','trafficking','smuggler'],
    'fraud':['fraud','fraudulent','scam','forgery','counterfeit','manufacture','scammer','fake','falsification'],
    'illegal possession':['possession','ban','10591'],
    'wanted':['wanted','manhunt'],
    'wrongful restraining': ['illegal detention','detention']
}

def analysis(tweet):
    words = word_tokenize(tweet)
    for prefix in word_map:
        match_score = 0
        for word in words:
            if word in word_map[prefix]:
                match_score += 1
                if match_score > 0:
                    return prefix
    return "non"

no_coords['type'] = no_coords['tweet'].apply(analysis)
no_coords[no_coords['type'] != 'non']

,tweet,coords,loc_name,region_name,type
0,pnp oic pltgen danao jr confirmed two members ...,124.386787/12.0354,danao sorsogon,Bicol Region (Region V),terrorism
3,three former members ctg masbate reintegrated ...,123.931534/11.18135,masbate masbate,Bicol Region (Region V),terrorism
4,37yearold drug suspect busted antiillegal drug...,123.642082/7.73305,san,Zamboanga Peninsula (Region IX),drug
7,police arrested monday member communist terror...,122.032257/17.68758,cagayan cagayan,Cagayan Valley (Region II),terrorism
11,municipallevel wanted person bohol finally arr...,124.433243/8.60917,bohol,Central Visayas (Region VII),wanted
...,...,...,...,...,...
14693,enrile martial law extension deter terrorism m...,121.99823/16.606159,enrile,Cagayan Valley (Region II),terrorism
14868,former customs commissioner faeldon released d...,121.477913/14.55693,rizal rizal,CALABARZON (Region IV-A),wrongful restraining
14969,bulacan five killed 95 nabbed antidrugs ops,121.157318/13.80879,bulacan bulacan,Central Luzon (Region III),murder
15227,look two different computerized facial composi...,123.107323/6.36558,mabuhay,Zamboanga Peninsula (Region IX),robbery


Model Training and Evaluation

In [76]:
from sklearn.model_selection import train_test_split
df = no_coords[['tweet', 'type', 'region_name', 'coords']].dropna()
X = df.tweet
y = df.type

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

print( "Train size: " + str(X_train.shape[0]))
print( "Test size: " + str(X_test.shape[0]))
print("\nTotal rows (Unsplit): " + str(df.shape[0]))
print("Total rows (Split): " + str(X_train.shape[0] + X_test.shape[0]))

Train size: 3490
Test size: 873

Total rows (Unsplit): 4363
Total rows (Split): 4363


In [77]:
# NB Classification
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)
# NB Metrics
from sklearn import metrics
predicted = text_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted))
# Show difference
pred_y_df=pd.DataFrame({'Text':X_test ,'Actual Value':y_test,'Predicted Value':predicted})
#pred_y_df[pred_y_df['Predicted Value'] == 'non']
pred_y_df

                      precision    recall  f1-score   support

                drug       0.88      0.31      0.46        45
               fraud       0.00      0.00      0.00         4
  illegal possession       0.00      0.00      0.00         3
          kidnapping       0.00      0.00      0.00         2
              murder       0.00      0.00      0.00        24
                 non       0.88      1.00      0.94       749
             robbery       0.00      0.00      0.00         7
           smuggling       0.00      0.00      0.00         4
           terrorism       1.00      0.32      0.49        31
              wanted       0.00      0.00      0.00         3
wrongful restraining       0.00      0.00      0.00         1

            accuracy                           0.89       873
           macro avg       0.25      0.15      0.17       873
        weighted avg       0.84      0.89      0.85       873



,Text,Actual Value,Predicted Value
1248,eleazar assures continuous reforms pnpa read g...,non,non
1353,isabela cop faces dismissal extorting money ar...,drug,non
7142,pals 2nd a350 jet arrives manila,non,non
1601,visit baguio city police office baguio city be...,non,non
7208,forced evacuation pangasinan 5 bulacan areas s...,non,non
...,...,...,...
4235,president duterte arrives papua new guinea fri...,non,non
6117,look san carlos city pangasinan convening cdrr...,non,non
11985,read president duterte calls forces wish see w...,non,non
9840,manila bulletin headline today 06052018 grab c...,non,non


In [78]:
# SVM Classification
from sklearn.linear_model import SGDClassifier
import pickle
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, n_iter_no_change=5, random_state=42))])
text_clf_svm = text_clf_svm.fit(X_train, y_train)

# SVM Metrics
from sklearn import metrics
predicted_svm = text_clf_svm.predict(X_test)
print(metrics.classification_report(y_test, predicted_svm))
# Show difference
pred_y_df=pd.DataFrame({'Text':X_test ,'Actual Value':y_test,'Predicted Value':predicted_svm})

pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(text_clf_svm, file)
pred_y_df

                      precision    recall  f1-score   support

                drug       0.95      0.82      0.88        45
               fraud       0.00      0.00      0.00         4
  illegal possession       0.00      0.00      0.00         3
          kidnapping       0.00      0.00      0.00         2
              murder       1.00      0.54      0.70        24
                 non       0.94      1.00      0.97       749
             robbery       0.00      0.00      0.00         7
           smuggling       0.00      0.00      0.00         4
           terrorism       0.96      0.71      0.81        31
              wanted       1.00      1.00      1.00         3
wrongful restraining       0.00      0.00      0.00         1

            accuracy                           0.94       873
           macro avg       0.44      0.37      0.40       873
        weighted avg       0.92      0.94      0.93       873



,Text,Actual Value,Predicted Value
1248,eleazar assures continuous reforms pnpa read g...,non,non
1353,isabela cop faces dismissal extorting money ar...,drug,drug
7142,pals 2nd a350 jet arrives manila,non,non
1601,visit baguio city police office baguio city be...,non,non
7208,forced evacuation pangasinan 5 bulacan areas s...,non,non
...,...,...,...
4235,president duterte arrives papua new guinea fri...,non,non
6117,look san carlos city pangasinan convening cdrr...,non,non
11985,read president duterte calls forces wish see w...,non,non
9840,manila bulletin headline today 06052018 grab c...,non,non


In [11]:
from sklearn.linear_model import LogisticRegression

text_clf_log = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression())])
text_clf_log = text_clf_log.fit(X_train, y_train)
# NB Metrics
from sklearn import metrics
predicted = text_clf_log.predict(X_test)
print(metrics.classification_report(y_test, predicted))
# Show difference
pred_y_df=pd.DataFrame({'Text':X_test ,'Actual Value':y_test,'Predicted Value':predicted})


              precision    recall  f1-score   support

  carnapping       0.00      0.00      0.00         1
        drug       0.64      1.00      0.78        41
       fraud       0.00      0.00      0.00         3
  kidnapping       0.00      0.00      0.00         1
      murder       1.00      0.27      0.43        11
        rape       0.00      0.00      0.00         1
     robbery       0.00      0.00      0.00         2
   smuggling       0.00      0.00      0.00         3
   terrorism       0.00      0.00      0.00         3
      wanted       0.00      0.00      0.00         1

    accuracy                           0.66        67
   macro avg       0.16      0.13      0.12        67
weighted avg       0.56      0.66      0.55        67



In [82]:
final_df.tweet

0        pnp oic pltgen danao jr confirmed two members ...
1        antiillegal drug operation also led seizure 25...
2        regional level wanted person illegal drugs nab...
3        three former members ctg masbate reintegrated ...
4        37yearold drug suspect busted antiillegal drug...
                               ...                        
15326        deped strives enhance reading skills students
15327    ovp taking iso certification challenge improve...
15328    shooting stemmed heated argument isagani anche...
15329       afp engineers clear 30 marawi areas explosives
15330    10 us citizens 2 locals killed costa rica plan...
Name: tweet, Length: 15331, dtype: object

In [83]:
pred_y_df=pd.DataFrame({'tweet':final_df.tweet ,'type':text_clf_svm.predict(final_df.tweet)})
pred_y_df[pred_y_df['type'] != 'non']

,tweet,type
1,antiillegal drug operation also led seizure 25...,drug
2,regional level wanted person illegal drugs nab...,drug
4,37yearold drug suspect busted antiillegal drug...,drug
10,police busted illegal drug activities female d...,drug
11,municipallevel wanted person bohol finally arr...,wanted
...,...,...
14969,bulacan five killed 95 nabbed antidrugs ops,murder
15000,pnp cordillera 2017 2293 wanted persons arrested,wanted
15155,backpackers critically ill australia mass drug...,drug
15276,iranian protesters attack police stations leas...,murder


Heat Map

In [65]:
specific_df = df[df['type'] != 'non'].reset_index()[['coords','type']]
type = "wanted"
if type != "all":
    specific_df = specific_df[specific_df['type'] == type].reset_index()[['coords','type']]
specific_df

,coords,type
0,124.499069/8.69807,wanted
1,121.806137/6.02781,wanted
2,123.550583/8.46034,wanted
3,125.003799/7.65322,wanted
4,125.523331/11.77796,wanted
5,123.570129/7.85836,wanted
6,123.511253/7.96449,wanted
7,120.637497/15.86502,wanted
8,125.497231/11.90894,wanted
9,122.320763/13.10026,wanted


In [66]:
# Heat map test
import folium
from folium.plugins import HeatMap

heat_list = []
for _loc in range(specific_df.shape[0]):
#for _loc in range(10):
    temp = specific_df.coords[_loc].split("/")
    heat_list.append([float(temp[1]), float(temp[0]), 1])
#heat_list = [[11.33877, 124.552177, 0.8]]

mapObj = folium.Map([12, 122], zoom_start=6)
gradient={'0.0':'Navy', '0.25':'Blue','0.5':'Green', '0.75':'Yellow','1': 'Red'}
#HeatMap(data=heat_list, gradient=gradient, radius=25, blur = 10, min_opacity = 0.25, max_val = 0.0005).add_to(mapObj)
HeatMap(heat_list,radius=25, max_zoom=13).add_to(mapObj)
mapObj.save("output.html")

Plot per region

In [14]:
pred =text_clf_svm.predict(df.tweet)
len(pred)
pred_df =pd.DataFrame({'tweet':df.tweet ,'type':pred, 'region': df.region_name})
pred_df[['type', 'region']].to_csv("crimes.csv")

In [95]:
regio = pred_df.region.unique()
region_list = []
type_list = []
percent_list = []
others_list = []
for reg in regio:
    temp  = pred_df[pred_df.region == reg].type.value_counts()
    region_list.append(reg)
    type_list.append(temp.index[0])
    percent_list.append(temp[0] / temp.sum() * 100)
    word = ""
    for item in temp.index:
        word += str(item) + ", "
    others_list.append(word)

chloro_df = pd.DataFrame({'Region Name':region_list ,'Most Likely Crime':type_list, 'Probability Percent of Most Likely Crime':percent_list, 'All Crimes':others_list})
chloro_df

,Region Name,Most Likely Crime,Probability Percent of Most Likely Crime,All Crimes
0,Bicol Region (Region V),drug,46.666667,"drug, terrorism, illegal possession, murder, k..."
1,Zamboanga Peninsula (Region IX),drug,58.333333,"drug, smuggling, wanted, robbery, terrorism,"
2,Cagayan Valley (Region II),drug,75.000000,"drug, terrorism, murder,"
3,Central Visayas (Region VII),drug,81.395349,"drug, wanted, murder, kidnapping, rape, terror..."
4,Autonomous Region of Muslim Mindanao (ARMM),drug,35.294118,"drug, murder, terrorism, wanted, illegal posse..."
5,Western Visayas (Region VI),drug,60.000000,"drug, terrorism, murder, wanted, smuggling,"
6,MIMAROPA (Region IV-B),drug,100.000000,"drug,"
7,Cordillera Administrative Region (CAR),drug,96.153846,"drug, terrorism,"
8,CALABARZON (Region IV-A),drug,72.727273,"drug, murder, kidnapping, wanted, smuggling, r..."
9,Eastern Visayas (Region VIII),drug,43.478261,"drug, terrorism, wanted, rape, kidnapping, mur..."


In [96]:
import plotly.express as px

fig = px.choropleth_mapbox(
    chloro_df,
    locations="Region Name",
    geojson=regions,
    color="Probability Percent of Most Likely Crime",
    featureidkey="properties.REGION",
    hover_name="Region Name",
    hover_data=["All Crimes", "Most Likely Crime"],
    mapbox_style="carto-positron",
    center={"lat": 12, "lon": 122},
    zoom=5,
)

fig.write_html("file.html")

In [79]:
final_df

,index,tweet,coords
0,0,pnp oic pltgen danao jr confirmed two members ...,NaN
1,1,antiillegal drug operation also led seizure 25...,NaN
2,2,regional level wanted person illegal drugs nab...,NaN
3,3,three former members ctg masbate reintegrated ...,NaN
4,4,37yearold drug suspect busted antiillegal drug...,NaN
...,...,...,...
15326,6506,deped strives enhance reading skills students,NaN
15327,6507,ovp taking iso certification challenge improve...,NaN
15328,6508,shooting stemmed heated argument isagani anche...,NaN
15329,6509,afp engineers clear 30 marawi areas explosives,NaN
